In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
warnings.filterwarnings("ignore")

In [4]:
loan_data_file = pd.read_csv("loan_data_2007_2014.csv", header = 0, index_col = 0)

In [5]:
#Shape of the original dataset
loan_data_file.shape

(466285, 73)

In [6]:
loan_data_file.head(5)

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
id,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
loan_data_file.columns

Index(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
       'ac

In [8]:
loan_data_file['loan_status'].value_counts(normalize=True)

Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: loan_status, dtype: float64

Label the target variable 'loan_status' to 0 and 1. 0 stands for Non-fraud, 1 stands for fraud.

In [9]:

# create a new column based on the loan_status column that will be our target variable
loan_data_file['good_bad'] = np.where(loan_data_file.loc[:, 'loan_status'].isin(['Charged Off', 'Default',
                                                                       'Late (31-120 days)',
                                                                       'Does not meet the credit policy. Status:Charged Off']), 0, 1)

### 2. Data cleaning and Pre-processing

##### 2.1 Remove irrelavant columns (redundant and forward looking columns) mentioned below:
'member_id', 'emp_title', 'pymnt_plan', 'url', 'desc', 'zip_code', 'addr_state', 'mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m'

In [10]:
# drop columns with more than 80% null values
loan_data_file.dropna(thresh = loan_data_file.shape[0]*0.2, how = 'all', axis = 1, inplace = True)

In [11]:
loan_data_file.drop(columns =['member_id', 'emp_title', 'pymnt_plan', 'url', 'desc', 
                         'title','zip_code', 'next_pymnt_d', 'recoveries', 'collection_recovery_fee','addr_state', 'mths_since_last_delinq', 
                         'next_pymnt_d', 'tot_coll_amt', 'tot_cur_bal', 'mths_since_last_major_derog'], axis = 1, inplace = True)

In [12]:
# shape of dataset after dropping irrelevant columns
loan_data_file.shape

(466285, 41)

In [13]:
# split data into 80/20 while keeping the distribution of bad loans in test set same as that in the pre-split dataset
X = loan_data_file.drop('good_bad', axis = 1)
y = loan_data_file['good_bad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 42, stratify = y)

# hard copy the X datasets to avoid Pandas' SetttingWithCopyWarning when we play around with this data later on.
# this is currently an open issue between Pandas and Scikit-Learn teams
X_train, X_test = X_train.copy(), X_test.copy()

Other Data cleaning tasks that are required as follows (Ref: https://github.com/finlytics-hub/credit_risk_model/blob/master/Credit_Risk_Model_and_Credit_Scorecard.ipynb)

*   Remove text from the emp_length column (e.g., years) and convert it to numeric
*   For all columns with dates: convert them to Python’s datetime format, create a new column as a difference between model development date and the respective date feature and then drop the original feature
*   Remove text from the term column and convert it to numeric


 2.3.1 Emp_Length
 - Impute nan values in 'emp_length'

In [14]:
'''function to clean up the emp_length column, assign 0 to NANs, and convert to numeric
'''
def emp_length_converter(df, column):
    df[column] = df[column].str.replace('\+ years', '')
    df[column] = df[column].str.replace('< 1 year', str(0))
    df[column] = df[column].str.replace(' years', '')
    df[column] = df[column].str.replace(' year', '')
    df[column] = pd.to_numeric(df[column])
    df[column].fillna(value = 0, inplace = True)

2.3.2 Date Columns: transform to datetime format.

In [15]:
'''
function to convert date columns to datetime format and
create a new column as a difference between today and the respective date
'''
def date_columns(df, column):
    # store current month
    today_date = pd.to_datetime('2020-08-01')
    # convert to datetime format
    df[column] = pd.to_datetime(df[column], format = "%b-%y")
    # calculate the difference in months and add to a new column
    df['mths_since_' + column] = round(pd.to_numeric((today_date - df[column]) / 
							np.timedelta64(1, 'M')))
    # make any resulting -ve values to be equal to the max date
    df['mths_since_' + column] = df['mths_since_' + column].apply(
		lambda x: df['mths_since_' + column].max() if x < 0 else x)
    # drop the original date column
    df.drop(columns = [column], inplace = True)

In [16]:
'''
function to remove 'months' string from the 'term' column and convert it to numeric
'''
def loan_term_converter(df, column):
    df[column] = pd.to_numeric(df[column].str.replace(' months', ''))

In [17]:
#apply to X_train
emp_length_converter(X_train,'emp_length')
date_columns(X_train, 'earliest_cr_line')
date_columns(X_train, 'issue_d')
date_columns(X_train, 'last_pymnt_d')
date_columns(X_train, 'last_credit_pull_d')
loan_term_converter(X_train, 'term')

In [18]:
X_train.head(5)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,total_rev_hi_lim,mths_since_earliest_cr_line,mths_since_issue_d,mths_since_last_pymnt_d,mths_since_last_credit_pull_d
id,,,,,,,,,,,,,,,,,,,,,
10588532,15000,15000,15000.0,36,8.90,476.30,A,A5,7.0,MORTGAGE,...,8940.22,0.0,1,INDIVIDUAL,0.0,35100.0,296.0,79.0,62.0,55.0
11215587,8000,8000,8000.0,60,18.25,204.24,D,D3,10.0,OWN,...,204.24,0.0,1,INDIVIDUAL,0.0,41300.0,301.0,79.0,55.0,55.0
14218271,12150,12150,12100.0,60,18.92,314.65,D,D4,3.0,OWN,...,314.65,0.0,1,INDIVIDUAL,0.0,9800.0,359.0,75.0,55.0,55.0
6504869,10000,10000,10000.0,36,6.03,304.36,A,A1,3.0,MORTGAGE,...,3553.37,0.0,1,INDIVIDUAL,0.0,10100.0,224.0,84.0,59.0,59.0
3929378,15825,15825,15825.0,36,12.12,526.53,B,B3,10.0,MORTGAGE,...,526.53,0.0,1,INDIVIDUAL,0.0,32400.0,494.0,87.0,78.0,55.0


In [19]:
# update the test data with all functions defined so far
emp_length_converter(X_test, 'emp_length')
date_columns(X_test, 'earliest_cr_line')
date_columns(X_test, 'issue_d')
date_columns(X_test, 'last_pymnt_d')
date_columns(X_test, 'last_credit_pull_d')
loan_term_converter(X_test, 'term')

In [20]:
X_train.fillna(0, inplace=True)

In [21]:
X_test.fillna(0, inplace=True)

In [22]:
train = pd.concat([y_train, X_train], axis=1)
test = pd.concat([y_test, X_test], axis=1)
train.to_csv('TrainData.csv',index=False)
test.to_csv('TestData.csv',index=False)